In [13]:
import sys, os, random

import numpy as np
import pandas as pd

import networkx as nx
import networkx.algorithms.components.connected as nxacc
import networkx.algorithms.dag as nxadag


import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
class LoadOntology():
    """
    Loads VNN's hierarchy as DiGraph, as well as other helpful mappings used throughout VNN code. 
    
    Specify alternate ontology during initialization with "ontology_file"
    
    Attributes:
    * self.ont_filepath --> filepath to the ontology file used
    * self.genes --> list of all genes used in VNN model
    * self.dG --> networkx.DiGraph object of the ontology. Genes will need to be added separately if desired (not needed to run VNN)
    * self.root --> str, the name of the network's root
    * self.term_size_map --> dict with {"term":n_neurons} used in model architecture
    * self.term2genes --> dict with {"term":[genes]} for all genes connected to that term
    * self.gene2terms --> dict with {"gene":[terms]} for all terms connected to that gene
    * self.term2layer --> dict with {"term":layer_number} for all GO terms in dG. Root is layer 1, then increments up the hierarchy  
    * self.layer2terms --> dict with {layer_number:[terms]} for all terms in that layer number
    * self.all --> tuple of [dG, root, term_size_map, term_direct_gene_map, gene_2_term], which can be unpacked directly into VNN
    """
    def __init__(self, ontology_file='ont.txt'):
        self.data_dir = os.path.abspath('../data')
        print('data dir', self.data_dir)
        
        self.ont_filepath = os.path.join(self.data_dir, ontology_file)

        self.genes = self.load_genes()
        
        dG, root, term_size_map, term_direct_gene_map, gene_2_term = self.load()
        
        self.dG = dG
        self.root = root
        self.term_size_map = term_size_map
        self.term2genes = term_direct_gene_map
        self.gene2terms = gene_2_term
        
        term2layer, layer2terms = self.make_layer_maps()
        self.term2layer = term2layer
        self.layer2terms = layer2terms
        
        self.go2name = self.load_go_descriptions()
        
        self.all = (dG, root, term_size_map, term_direct_gene_map, gene_2_term)
    
    def load_genes(self):
        filepath = os.path.join(self.data_dir, 'gene2ind.txt')
        df = pd.read_csv(filepath, sep='\t', names=['ind','gene'])
        return [x for x in df['gene']]
    
    def load(self):
        
        dG = nx.DiGraph()
        gene_2_term = {}
        term_direct_gene_map = {}
        term_size_map = {}

        with open(self.ont_filepath, 'r') as file_handle:
            gene_set = set()
            for line in file_handle:
                line = line.rstrip().split()

                if line[2] == 'default':
                    dG.add_edge(line[0], line[1])
                else:
                    if line[1] not in self.genes:
                        continue

                    if line[0] not in term_direct_gene_map:
                        term_direct_gene_map[line[0]] = set()

                    if line[1] not in gene_2_term:
                        gene_2_term[line[1]] = set()

                    # Collect the "child" of genes at position 0
                    term_direct_gene_map[line[0]].add(line[1])

                    gene_2_term[line[1]].add(line[0])

                    gene_set.add(line[1])

            print('There are', len(gene_set), 'genes')

        for term in dG.nodes():
            term_gene_set = set()
            if term in term_direct_gene_map:
                term_gene_set = term_direct_gene_map[term]

            deslist = nxadag.descendants(dG, term)

            for child in deslist:
                if child in term_direct_gene_map:
                    term_gene_set = term_gene_set | term_direct_gene_map[child]

            if len(term_gene_set) == 0:
                print('There are empty terms, please delete term:', term)
                sys.exit(1)
            else:
                term_size_map[term] = len(term_gene_set)

        leaves = [n for n in dG.nodes if dG.in_degree(n) == 0]

        root = leaves[0]

        uG = dG.to_undirected()
        connected_subG_list = list(nxacc.connected_components(uG))

        print('There are', len(leaves), 'roots:', leaves[0])
        print('There are', len(dG.nodes()), 'terms')
        print('There are', len(connected_subG_list), 'connected componenets')

        if len(leaves) > 1:
            print(
                'There are more than 1 root of ontology. Please use only one root.')
            sys.exit(1)
        if len(connected_subG_list) > 1:
            print('There are more than connected components. Please connect them.')
            sys.exit(1)

        return dG, root, term_size_map, term_direct_gene_map, gene_2_term
    
    def make_layer_maps(self):
        dGl = self.dG.copy()

        term2layer = {}
        layer2terms = {}
        i = 1
        while True:
            leaves = [n for n in dGl.nodes() if dGl.in_degree(n) == 0]

            if len(leaves) == 0:
                break

            # leaves are only terms connected to genes
            layer2terms[i] = leaves

            for term in leaves:
                term2layer[term] = i

            i += 1

            dGl.remove_nodes_from(leaves)
            
        return term2layer, layer2terms
    
    def load_go_descriptions(self):
        go_2_name_file = '/cellar/users/pgwall/DrugCell/data/go_term_descriptions.txt'
        go_2_name_df = pd.read_csv(go_2_name_file, sep='\t', names=['def', 'name', 'term'], skiprows=1)

        go_2_name_file_2 = '/cellar/users/pgwall/DrugCell/Models/super_model/nest_vnn/validation/go_term_annotations.tsv'
        go_2_name_df_2 = pd.read_csv(go_2_name_file_2, sep='\t', names=['term','name','process','def'])

        go_2_name = dict(zip(go_2_name_df['term'].tolist(), go_2_name_df['name'].tolist()))
        go_2_name_2 = dict(zip(go_2_name_df_2['term'].tolist(), go_2_name_df_2['name'].tolist()))

        go_2_name.update(go_2_name_2)
    
        return go_2_name
    
    def add_genes(self):
        """
        Returns copy of self.dG annotated with self.genes 
        """
        dGg = self.dG.copy()

        for gene, terms in self.gene2terms.items():
            for term in terms:
                dGg.add_edge(term, gene)
                
        return dGg
    
    def add_features(self, dG_genes, features):
        """
        Returns copy of dG_genes annotated with features
        """
        dGgf= dG_genes.copy()
        
        if not isinstance(features, list):
            features = [features]
            
        gene_nodes = [x for x in dGgf.nodes() if x in self.gene2terms]

        for gene in gene_nodes:
            for feat in features:
                feat_str = '_'.join([gene, feat])
                dGgf.add_edge(gene, feat_str)
                
        return dGgf

In [21]:
ont = LoadOntology()

data dir /cellar/users/pgwall/data


FileNotFoundError: [Errno 2] No such file or directory: '/cellar/users/pgwall/data/gene2ind.txt'

In [22]:
os.getcwd()

'/cellar/users/pgwall/qms'

In [23]:
os.path.abspath('../data')

'/cellar/users/pgwall/data'